In [131]:
# standard library imports
import csv
import datetime as dt
import json
import os
import statistics
import time
import tqdm

# third-party imports
import numpy as np
import pandas as pd
import requests

# environment variables
from dotenv import load_dotenv
load_dotenv()
ANYDEALS_API_KEY = os.getenv("ANYDEAL_KEY")
# customisations - ensure tables show all columns
# pd.set_option("max_columns", 100)
print(ANYDEALS_API_KEY)

d0754faa36ea0d0d87c737908e13de1b56e857e5


In [45]:
def get_request(url, parameters=None):
    """Return json-formatted response of a get request using optional parameters.
    
    Parameters
    ----------
    url : string
    parameters : {'parameter': 'value'}
        parameters to pass as part of get request
    
    Returns
    -------
    json_data
        json-formatted response (dict-like)
    """
    try:
        response = requests.get(url=url, params=parameters)
    except SSLError as s:
        print('SSL Error:', s)
        
        for i in range(5, 0, -1):
            print('\rWaiting... ({})'.format(i), end='')
            time.sleep(1)
        print('\rRetrying.' + ' '*10)
        
        # recusively try again
        return get_request(url, parameters)
    
    if response:
        return response.json()
    else:
        # response is none usually means too many requests. Wait and try again 
        print('No response, waiting 10 seconds...')
        time.sleep(10)
        print('Retrying.')
        return get_request(url, parameters)

In [47]:
url = "https://steamspy.com/api.php"


# request 'all' from steam spy and parse into dataframe
json_data = {}

# get the first 3 pages of data, 3000 games in total
# steamspy lists 1000 games per page, sorted by popularity
for i in range(0,3):
    parameters = {'request': 'all', 'page': i}
    json_data.update(get_request(url, parameters))
    time.sleep(60) # sleep for 60 seconds to avoid rate limiting
steam_spy_all = pd.DataFrame.from_dict(json_data, orient='index')
steam_spy_all.to_csv('steam_spy_all_3000.csv')
# generate sorted app_list from steamspy data
app_list = steam_spy_all[['appid', 'name']].reset_index(drop=True)

# export disabled to keep consistency across download sessions
app_list.to_csv('app_list.csv', index=False)

# instead read from stored csv
app_list = pd.read_csv('app_list.csv')

# display first few rows
app_list.head()

,appid,name
0,570,Dota 2
1,730,Counter-Strike: Global Offensive
2,578080,PUBG: BATTLEGROUNDS
3,1623730,Palworld
4,440,Team Fortress 2


In [13]:
# get data for each appid from isthereanydeal.com
def get_shop_data():
    """Return json-formatted response of a get request using optional parameters.
    
    Parameters
    ----------
    appid : int
        appid of game to get data for
    
    Returns
    -------
    json_data
        json-formatted response (dict-like)
    """
    url = f"https://api.isthereanydeal.com/service/shops/v1"
    parameters = {"key": ANYDEALS_API_KEY}
    
    response = requests.get(url=url, params=parameters)
    return response.json()

In [51]:
shop_info = pd.DataFrame.from_dict(get_shop_data())

In [53]:
shop_info.to_csv('shop_info.csv', index=False)

In [176]:
def get_anydeal_id(appid):
    """
    Match the appid to the corresponding anydeal id, if it exists.
    return the mapping pair in a json.
    """
    url = f"https://api.isthereanydeal.com/games/lookup/v1"
    parameters = {'key': ANYDEALS_API_KEY,'appid': appid}
    # print(parameters)
    time.sleep(0.4) # sleep for 0.4 seconds before any request to avoid rate limiting
    response = requests.get(url=url, params=parameters)
    # print(response.json())
    return response.json()['game']['id']

def get_price_history(appid):
    """
    Get the price history of a game from a specific shop.
    """
    url = f"https://api.isthereanydeal.com/games/history/v2"
    parameters = {"key": ANYDEALS_API_KEY, 'id':appid, 'country':'US','since':'2000-01-01T00:00:00+01:00','shops':'61'}
    time.sleep(0.4)
    response = requests.get(url=url, params=parameters)
    return {'appid': appid, 'price_history': response.json()}

def get_anydeal_game_info(id):
    """
    Get the game info from anydeal using the steam id.
    """
    url = f"https://api.isthereanydeal.com/games/info/v2"
    id = get_anydeal_id(id)
    parameters = {'key': ANYDEALS_API_KEY,'id':id}
    time.sleep(0.4)
    response_info = requests.get(url=url, params=parameters)

    response_history = get_price_history(id)

    return [response_info.json(), response_history]

def get_batch_info_handler(app_list):
    """
    Create a master table of all games in the app_list, with their corresponding anydeal id.
    """
    master_json_list = []
    master_json_list_history = []
    for appid in app_list['appid']:
        print('now processing appid:', appid)
        master_json_list.append(get_anydeal_game_info(appid)[0])
        master_json_list_history.append(get_anydeal_game_info(appid)[1])
    return [master_json_list, master_json_list_history]

def create_master_game_info(batch_size, app_list,start, end, handler, filename):
    """
    Create a master table of all games in the app_list.
    As the API may fail at any point, we will write the table to local storage after each batch.
    Each batch is converted to a list of jsons. We write the list into a txt file.
    Any time a batch is processed, we will write it to local file.
    Any time a batch fails, we will read the local file and continue from the last batch.


    batch_size: the number of games to process at once
    app_list: the list of games to process, in the form of a pandas dataframe
    start: the index to start processing from
    end: the index to end processing at
    handler: the function to process the batch
    filename: the name of the file to write to
    """

    for i in tqdm.tqdm(range(start, end, batch_size)):
        try:
            # handle the game info
            print(f"Processing batch {i} to {i+batch_size}")
            batch = app_list.iloc[i:i+batch_size]
            batch_json = handler(batch)[0]
            batch_df = pd.DataFrame.from_dict(batch_json)
            if i == 0:
                # batch_df.to_csv(filename, index=False)
                batch_df.to_csv(f'textfiles/game_info/last_successful_batch_{i}_'+filename, index=False)
            else:
                # batch_df.to_csv(filename, mode='a', header= False,index=False)
                batch_df.to_csv(f'textfiles/game_info/last_successful_batch_{i}_'+filename, index=False)
            print(f"Batch {i} to {i+batch_size} processed successfully.")
            # print("hello")
            # handle the price history
            print(f"PRICE HISTORY: Processing batch {i} to {i+batch_size}")
            batch_json_history = handler(batch)[1]
            batch_df_history = pd.DataFrame.from_dict(batch_json_history)
            if i == 0:
                # batch_df_history.to_csv('price_history.csv', index=False)
                batch_df_history.to_csv(f'textfiles/price_history/last_successful_batch_{i}_price_history.csv', index=False)
            else:
                # batch_df_history.to_csv('price_history.csv', mode='a',header=False, index=False)
                batch_df_history.to_csv(f'textfiles/price_history/last_successful_batch_{i}_price_history.csv', index=False)

            
        except Exception as e:
            print(f"Batch {i} to {i+batch_size} failed.")
            print(e)


In [183]:
app_all = pd.read_csv('app_list.csv')
create_master_game_info(10, app_all, 2700, len(app_all), get_batch_info_handler, 'game_info_anydeal.csv')

  0%|          | 0/30 [00:00<?, ?it/s]

Processing batch 2700 to 2710
Batch 2700 to 2710 processed successfully.
PRICE HISTORY: Processing batch 2700 to 2710


  3%|▎         | 1/30 [01:28<42:54, 88.79s/it]

Processing batch 2710 to 2720
Batch 2710 to 2720 processed successfully.
PRICE HISTORY: Processing batch 2710 to 2720


  7%|▋         | 2/30 [02:52<40:03, 85.83s/it]

...

 93%|█████████▎| 28/30 [50:17<02:41, 80.70s/it]

Processing batch 2980 to 2990
Batch 2980 to 2990 processed successfully.
PRICE HISTORY: Processing batch 2980 to 2990


 97%|█████████▋| 29/30 [51:38<01:20, 80.89s/it]

Processing batch 2990 to 3000
Batch 2990 to 3000 processed successfully.
PRICE HISTORY: Processing batch 2990 to 3000


100%|██████████| 30/30 [52:52<00:00, 105.76s/it]


In [184]:
price_history = pd.read_csv('price_history.csv')
price_history

,appid,price_history
0,018d937f-19a5-7057-bb6d-314d586e6dbc,"[{'timestamp': '2024-02-11T01:47:46+01:00', 's..."
1,018d937f-7851-7004-b780-3f657a301f9a,"[{'timestamp': '2024-02-11T01:47:46+01:00', 's..."
2,018d937f-57ce-723f-b7e7-7c7b5df93471,"[{'timestamp': '2024-02-11T03:51:23+01:00', 's..."
3,018d937f-50c1-7086-807c-e020c98c72b2,"[{'timestamp': '2024-07-18T19:16:16+02:00', 's..."
4,018d937e-fde4-72ff-a7af-45e4955a8dd6,"[{'timestamp': '2024-02-11T01:47:46+01:00', 's..."
...,...,...
3034,018d937e-f973-71db-988b-4b9d2c183d55,"[{'timestamp': '2024-07-25T19:16:13+02:00', 's..."
3035,018d96f1-a1a9-70ec-9c8a-06ef9eb86465,"[{'timestamp': '2024-02-11T08:58:04+01:00', 's..."
3036,018d979a-4ad7-71e4-aecc-eb83bede66c6,"[{'timestamp': '2024-02-11T02:18:13+01:00', 's..."
3037,018d937f-33c1-70d1-b6c0-2e871940149a,"[{'timestamp': '2024-07-22T19:16:52+02:00', 's..."


In [200]:
price_history['appid'].duplicated().sum()

0

In [194]:
game_info_anydeal = pd.read_csv('game_info_anydeal.csv')
game_info_anydeal.drop_duplicates(inplace=True)

In [199]:
game_info_anydeal.shape

(2994, 18)

In [138]:
get_all_price_history(pd.read_csv('app_list.csv').head(5))

KeyError: 'id'

In [136]:

price_items = {
    'first_available_price': '',
    'first_available_date': '',
    'historical_low': '',
    'historical_low_date': '',
    'historical_high_discount': '',
    'historical_high_discount_date': '',
    'first_discount': '',
    'first_discount_date': '',
    'last_discount': '',
    'last_discount_date': '',
    'average_duration_between_discounts_12months': '',
    'average_duration_of_discounts_12months': '',
    'avg_price': '',
    'avg_discount': '',
    # price decay is the average second derivative of a moving average of the price
    # we take 12 months of data, and calculate the moving average of the price
    # then we calculate the second derivative of the moving average
    # finally we average the second derivative over the 12 months
    'price_decay_rate': '', 

    # testing modules
}

# get price history for a game
anydeal_id = '018d937e-f4d4-72e7-8573-f89aa24ca824'
history = pd.DataFrame.from_dict(get_price_history(anydeal_id))
history

,timestamp,shop,deal
0,2024-07-22T19:16:37+02:00,"{'id': 61, 'name': 'Steam'}","{'price': {'amount': 4.99, 'amountInt': 499, '..."
1,2024-06-27T19:23:58+02:00,"{'id': 61, 'name': 'Steam'}","{'price': {'amount': 1.74, 'amountInt': 174, '..."
2,2024-04-22T19:16:52+02:00,"{'id': 61, 'name': 'Steam'}","{'price': {'amount': 4.99, 'amountInt': 499, '..."
3,2024-04-15T19:21:23+02:00,"{'id': 61, 'name': 'Steam'}","{'price': {'amount': 1.74, 'amountInt': 174, '..."
4,2024-03-21T18:18:27+01:00,"{'id': 61, 'name': 'Steam'}","{'price': {'amount': 4.99, 'amountInt': 499, '..."
...,...,...,...
146,2013-04-04T19:06:09+02:00,"{'id': 61, 'name': 'Steam'}","{'price': {'amount': 5.24, 'amountInt': 524, '..."
147,2013-02-14T20:06:07+01:00,"{'id': 61, 'name': 'Steam'}","{'price': {'amount': 3.4, 'amountInt': 340, 'c..."
148,2012-12-20T21:25:27+01:00,"{'id': 61, 'name': 'Steam'}","{'price': {'amount': 4.99, 'amountInt': 499, '..."
149,2012-12-13T19:26:03+01:00,"{'id': 61, 'name': 'Steam'}","{'price': {'amount': 8.74, 'amountInt': 874, '..."


In [ ]:

# remove shop info, as we only look at steam
history.drop(columns=['shop'], inplace=True)

# extract price, regular price, discount
history['price'] = history['deal'].apply(lambda x: x['price']['amount'])
history['price'] = history['price'].astype(float)
history['regular'] = history['deal'].apply(lambda x: x['regular']['amount'])
history['regular'] = history['regular'].astype(float)
history['discount'] = history['deal'].apply(lambda x: x['cut'])
history['discount'] = history['discount'].astype(int)

# drop deal column
history.drop(columns=['deal'], inplace=True)


In [83]:
history

,timestamp,price,regular,discount
0,2024-07-22T19:16:37+02:00,4.99,4.99,0
1,2024-06-27T19:23:58+02:00,1.74,4.99,65
2,2024-04-22T19:16:52+02:00,4.99,4.99,0
3,2024-04-15T19:21:23+02:00,1.74,4.99,65
4,2024-03-21T18:18:27+01:00,4.99,4.99,0
...,...,...,...,...
146,2013-04-04T19:06:09+02:00,5.24,34.99,85
147,2013-02-14T20:06:07+01:00,3.40,9.99,66
148,2012-12-20T21:25:27+01:00,4.99,9.99,50
149,2012-12-13T19:26:03+01:00,8.74,34.99,75


In [ ]:
def historical_price_transformer(anydeal_id):
    history = get_price_history(anydeal_id)